In [1]:
import os
import sys
import pandas as pd
from pathlib import Path
import numpy as np
from kmodes.kmodes import KModes
from kmodes.kprototypes import KPrototypes
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('./Engineered_Data/data.csv')

In [3]:
df

,AveragePaymentValue,CustomerKey,DueDate,EarliestPaymentDate,InvoiceKey,NumPayments,OriginalInvoiceAmount,LatestPaymentDate,PaymentType,TotalPaymentValue,RaisedDate,StatementTransactionType
0,168.000,16745090,2018-03-02,2018-11-23,195519971,1.0,168.00,2018-11-23,Barclays Multi,168.00,2018-01-03,INV
1,819.600,16743947,2018-02-03,2018-10-26,196442471,1.0,819.60,2018-10-26,Barclays Multi,819.60,2018-01-04,INV
2,718.200,16836264,2018-02-08,2018-11-05,196563186,1.0,749.40,2018-11-05,Barclays Multi,718.20,2018-01-09,INV
3,1873.080,16843107,2018-03-02,2018-09-04,196711370,1.0,1873.08,2018-09-04,Barclays Multi,1873.08,2018-01-11,INV
4,1316.090,16739410,2018-02-11,2018-09-26,197134638,1.0,1316.09,2018-09-26,Barclays Multi,1316.09,2018-01-12,INV
...,...,...,...,...,...,...,...,...,...,...,...,...
272237,24.815,16849111,2019-07-30,2019-07-30 00:00:00,196911580,2.0,49.63,2019-08-30 00:00:00,Barclays Multi,49.63,2019-06-30,INV
272238,178.165,16847651,2019-07-30,2019-09-24 00:00:00,195362531,2.0,356.33,2019-10-07 00:00:00,Barclays Multi,356.33,2019-06-30,INV
272239,36.310,16917308,2019-07-30,2019-07-29 00:00:00,196871858,2.0,72.62,2019-08-09 00:00:00,Barclays Multi,72.62,2019-06-30,INV
272240,311.440,16782671,2019-07-30,2019-07-22 00:00:00,195891626,2.0,311.44,2019-09-18 00:00:00,CheckM8,622.88,2019-06-30,INV


In [4]:
# Is this invoice paid fully ? (0/1 binary)
# just compare the original invoice amount with the total payment

df['IsInvoicePaidFully'] = df.apply(lambda row: 1 if row['TotalPaymentValue'] >= row['OriginalInvoiceAmount'] else 0, axis=1)


In [5]:
df

,AveragePaymentValue,CustomerKey,DueDate,EarliestPaymentDate,InvoiceKey,NumPayments,OriginalInvoiceAmount,LatestPaymentDate,PaymentType,TotalPaymentValue,RaisedDate,StatementTransactionType,IsInvoicePaidFully
0,168.000,16745090,2018-03-02,2018-11-23,195519971,1.0,168.00,2018-11-23,Barclays Multi,168.00,2018-01-03,INV,1
1,819.600,16743947,2018-02-03,2018-10-26,196442471,1.0,819.60,2018-10-26,Barclays Multi,819.60,2018-01-04,INV,1
2,718.200,16836264,2018-02-08,2018-11-05,196563186,1.0,749.40,2018-11-05,Barclays Multi,718.20,2018-01-09,INV,0
3,1873.080,16843107,2018-03-02,2018-09-04,196711370,1.0,1873.08,2018-09-04,Barclays Multi,1873.08,2018-01-11,INV,1
4,1316.090,16739410,2018-02-11,2018-09-26,197134638,1.0,1316.09,2018-09-26,Barclays Multi,1316.09,2018-01-12,INV,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
272237,24.815,16849111,2019-07-30,2019-07-30 00:00:00,196911580,2.0,49.63,2019-08-30 00:00:00,Barclays Multi,49.63,2019-06-30,INV,1
272238,178.165,16847651,2019-07-30,2019-09-24 00:00:00,195362531,2.0,356.33,2019-10-07 00:00:00,Barclays Multi,356.33,2019-06-30,INV,1
272239,36.310,16917308,2019-07-30,2019-07-29 00:00:00,196871858,2.0,72.62,2019-08-09 00:00:00,Barclays Multi,72.62,2019-06-30,INV,1
272240,311.440,16782671,2019-07-30,2019-07-22 00:00:00,195891626,2.0,311.44,2019-09-18 00:00:00,CheckM8,622.88,2019-06-30,INV,1


In [6]:
# Raised time to first payment
# difference between earliestpaymentdate and raised date

# convert to pd.datetime
df['RaisedDate'] =  pd.to_datetime(df['RaisedDate'])
df['DueDate'] =  pd.to_datetime(df['DueDate'])
df['EarliestPaymentDate'] =  pd.to_datetime(df['EarliestPaymentDate'])
df['LatestPaymentDate'] =  pd.to_datetime(df['LatestPaymentDate'])

df['RaisedToFirstPayment'] = (df['EarliestPaymentDate'] - df['RaisedDate']).dt.days

In [7]:
# Raised time to cleared(OriginalInvoiceAmount = Sum(PaymentValues)) in days
# if not cleared, -1, otherwise number of days until latest payment

df['RaisedToCleared'] = df.apply(lambda row: np.nan if row['IsInvoicePaidFully'] == 0 else (row['LatestPaymentDate'] - row['RaisedDate']).days, axis=1)

In [8]:
'''
Invoice cleared on time before or on DueDate (0/1) binary

paid fully and latest payment date <= duedate

'''

df['IsClearedOnTime'] = df.apply(lambda row: 1 if row['IsInvoicePaidFully'] == 1 and row['LatestPaymentDate'] <= row['DueDate'] else 0, axis=1)

In [9]:
df

,AveragePaymentValue,CustomerKey,DueDate,EarliestPaymentDate,InvoiceKey,NumPayments,OriginalInvoiceAmount,LatestPaymentDate,PaymentType,TotalPaymentValue,RaisedDate,StatementTransactionType,IsInvoicePaidFully,RaisedToFirstPayment,RaisedToCleared,IsClearedOnTime
0,168.000,16745090,2018-03-02,2018-11-23,195519971,1.0,168.00,2018-11-23,Barclays Multi,168.00,2018-01-03,INV,1,324,324.0,0
1,819.600,16743947,2018-02-03,2018-10-26,196442471,1.0,819.60,2018-10-26,Barclays Multi,819.60,2018-01-04,INV,1,295,295.0,0
2,718.200,16836264,2018-02-08,2018-11-05,196563186,1.0,749.40,2018-11-05,Barclays Multi,718.20,2018-01-09,INV,0,300,NaN,0
3,1873.080,16843107,2018-03-02,2018-09-04,196711370,1.0,1873.08,2018-09-04,Barclays Multi,1873.08,2018-01-11,INV,1,236,236.0,0
4,1316.090,16739410,2018-02-11,2018-09-26,197134638,1.0,1316.09,2018-09-26,Barclays Multi,1316.09,2018-01-12,INV,1,257,257.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272237,24.815,16849111,2019-07-30,2019-07-30,196911580,2.0,49.63,2019-08-30,Barclays Multi,49.63,2019-06-30,INV,1,30,61.0,0
272238,178.165,16847651,2019-07-30,2019-09-24,195362531,2.0,356.33,2019-10-07,Barclays Multi,356.33,2019-06-30,INV,1,86,99.0,0
272239,36.310,16917308,2019-07-30,2019-07-29,196871858,2.0,72.62,2019-08-09,Barclays Multi,72.62,2019-06-30,INV,1,29,40.0,0
272240,311.440,16782671,2019-07-30,2019-07-22,195891626,2.0,311.44,2019-09-18,CheckM8,622.88,2019-06-30,INV,1,22,80.0,0


In [10]:
df.to_csv(path_or_buf='./Engineered_Data/invoice_data.csv', index=False)